In [14]:
import rioxarray
import geopandas
import leafmap
import os
import datetime
from osgeo import gdal
from tqdm import tqdm
import cdsapi
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime
import rasterio
import pandas as pd
import pickle

In [9]:
grib_file = "era_5_evap_soil_moist.grib"
dest_file = "data/era_5_evap_soil_tiff/" + "era_5_evap_soil_moist.tiff"
dest_dir = "data/era_5_evap_soil_tiff"
grace_dates_dir = "data/grace_dates.pickle"

Method for downloading ERA-5 data. The data is downloaded from the Copernicus Climate Data Store (CDS) using the cdsapi package. The data is downloaded in grib format and saved to the specified directory (`grib_file`).
monthly_averaged_reanalysis means that data with the date label contains averaged data from the last 30 days up to this date.

In [20]:

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'grib',
        'variable': [
            'evaporation', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2',
            'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4',
        ],
        'year': [
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016',
            '2017', '2018', '2019',
            '2020', '2021', '2022',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'area': [
            51.17, 18.4, 50.47,
            19.68,
        ],
        'product_type': 'monthly_averaged_reanalysis',
        'time': '00:00',
    },
    
    grib_file)

2023-07-11 22:36:17,608 INFO Welcome to the CDS
2023-07-11 22:36:17,609 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-07-11 22:36:17,808 INFO Request is queued
2023-07-11 22:36:18,900 INFO Request is running
2023-07-11 23:04:44,459 INFO Request is completed
2023-07-11 23:04:44,461 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data6/adaptor.mars.internal-1689107778.3299725-31967-16-055edeb2-4462-4c2f-89be-4be3fb9be295.grib to era_5_evap_soil_moist.grib (295.3K)
2023-07-11 23:04:49,219 INFO Download rate 62.1K/s 


Result(content_length=302400,content_type=application/x-grib,location=https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data6/adaptor.mars.internal-1689107778.3299725-31967-16-055edeb2-4462-4c2f-89be-4be3fb9be295.grib)

In [10]:
grib_f = "data/" + grib_file

An example how to extract TIFFs from the grib. Below there are cells for extracting particular variables (evaporation, soil moisture).

In [ ]:
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)


src_ds = gdal.Open(grib_f)
bands = [] # Set up array for gdal.Translate(). 
if src_ds is not None:
    bandNum = src_ds.RasterCount # Get band count
for i in range(bandNum+1): # Update array based on band count
    if (i==0): #gdal starts band counts at 1, not 0 like the Python for loop does.
        pass
    else:
        bands.append(i)

# Open output format driver
out_form= "GTiff"

# Output to new format using gdal.Translate. See https://gdal.org/python/ for osgeo.gdal.Translate options.
dst_ds = gdal.Translate(dest_file, src_ds, format=out_form, bandList=bands)

# Properly close the datasets to flush to disk
dst_ds = None
src_ds = None

In [15]:
ds = gdal.Open(grib_f)
rasterBand = ds.GetRasterBand(2)
meta = rasterBand.GetMetadata()
dt = datetime.datetime.fromtimestamp(int(meta['GRIB_VALID_TIME'].lstrip()[:-8]))
meta

{'GRIB_COMMENT': 'Volumetric soil water layer 1 [m^3/m^3]',
 'GRIB_ELEMENT': 'SWVL1',
 'GRIB_FORECAST_SECONDS': '0',
 'GRIB_REF_TIME': '1009843200',
 'GRIB_SHORT_NAME': '0-7-DBLY',
 'GRIB_UNIT': '[m^3/m^3]',
 'GRIB_VALID_TIME': '1009843200'}

In [16]:
def get_file_title(element):
    if element.find("E") != -1:
        return "evaporation/" + element + "_"
    if element.find("SWVL1") != -1:
        return "vol_soil_lvl_1/" + element + "_"
    if element.find("SWVL2") != -1:
        return "vol_soil_lvl_2/" + element + "_"
    if element.find("SWVL3") != -1:
        return "vol_soil_lvl_3/" + element + "_"
    if element.find("SWVL4") != -1:
        return "vol_soil_lvl_4/" + element + "_"

In [17]:
def prepare_datetime(date):
    if date.strftime("%d") != "01":
        date = date + datetime.timedelta(days=1)
    return date.strftime("%Y-%m-%d")

In [18]:
if not os.path.exists("data/era_5_evap_soil_tiff/evaporation"):
    os.makedirs("data/era_5_evap_soil_tiff/evaporation")

if not os.path.exists("data/era_5_evap_soil_tiff/vol_soil_lvl_1"):
    os.makedirs("data/era_5_evap_soil_tiff/vol_soil_lvl_1")

if not os.path.exists("data/era_5_evap_soil_tiff/vol_soil_lvl_2"):
    os.makedirs("data/era_5_evap_soil_tiff/vol_soil_lvl_2")

if not os.path.exists("data/era_5_evap_soil_tiff/vol_soil_lvl_3"):
    os.makedirs("data/era_5_evap_soil_tiff/vol_soil_lvl_3")

if not os.path.exists("data/era_5_evap_soil_tiff/vol_soil_lvl_4"):
    os.makedirs("data/era_5_evap_soil_tiff/vol_soil_lvl_4")

out_form= "GTiff"
for i in range(1, ds.RasterCount+1):
    rasterBand = ds.GetRasterBand(i)
    meta = rasterBand.GetMetadata()
    dt = prepare_datetime(datetime.datetime.fromtimestamp(int(meta['GRIB_VALID_TIME'].lstrip()[:-8])))
    filename = get_file_title(meta['GRIB_ELEMENT']) + dt + ".tif"
    filename = os.path.join(dest_dir, filename)
    dst_ds = gdal.Translate(filename, ds, format=out_form, bandList=[i])
    dst_ds = None
    

Exportation of the particular data to pandas dataframes. Data format: {date, mask (array 8x13 pixels)}.

In [23]:
aoi_dir = "data/aoi.geojson"
geodf = geopandas.read_file(aoi_dir)

grace_dates - file necessary to read data only from dates with grace measurements. 

In [24]:
with open(grace_dates_dir, 'rb') as f:
    grace_dates = pickle.load(f)

grace_dates = [datetime.datetime.strptime(date, "%Y-%m-%d") for date in grace_dates]

In [25]:
def find_year_and_month_datetime(dt, dates_list):
    dt = datetime.datetime.strptime(dt, '%Y-%m-%d')
    for date in dates_list:
        if dt.year == date.year and dt.month == date.month:
            return date
    return None

In [26]:
evaporation_dir = os.path.join(dest_dir, "evaporation")
file_list = os.listdir(evaporation_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(evaporation_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    dt = f[2:-4]
    date = find_year_and_month_datetime(dt, grace_dates)
    if date is not None:
        df = df.append({'date': date.strftime("%Y-%m-%d"), 'value': masked.data}, ignore_index=True)
        if date.year == 2012 and date.month == 1:
            df = df.append({'date': date.strftime("%Y-%m") + "-16", 'value': masked.data}, ignore_index=True)
        if date.year == 2015 and date.month == 4:
            df = df.append({'date': "2015-04-27", 'value': masked.data}, ignore_index=True)
    
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_evaporation.pickle")

In [27]:
vol_soil_lvl_1_dir = os.path.join(dest_dir, "vol_soil_lvl_1")
file_list = os.listdir(vol_soil_lvl_1_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(vol_soil_lvl_1_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    dt = f[6:-4]
    date = find_year_and_month_datetime(dt, grace_dates)
    if date is not None:
        df = df.append({'date': date.strftime("%Y-%m-%d"), 'value': masked.data}, ignore_index=True)
        if date.year == 2012 and date.month == 1:
            df = df.append({'date': date.strftime("%Y-%m") + "-16", 'value': masked.data}, ignore_index=True)
        if date.year == 2015 and date.month == 4:
            df = df.append({'date': "2015-04-27", 'value': masked.data}, ignore_index=True)
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_vol_soil_lvl_1.pickle")

In [28]:
vol_soil_lvl_2_dir = os.path.join(dest_dir, "vol_soil_lvl_2")
file_list = os.listdir(vol_soil_lvl_2_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(vol_soil_lvl_2_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    dt = f[6:-4]
    date = find_year_and_month_datetime(dt, grace_dates)
    if date is not None:
        df = df.append({'date': date.strftime("%Y-%m-%d"), 'value': masked.data}, ignore_index=True)
        if date.year == 2012 and date.month == 1:
            df = df.append({'date': date.strftime("%Y-%m") + "-16", 'value': masked.data}, ignore_index=True)
        if date.year == 2015 and date.month == 4:
            df = df.append({'date': "2015-04-27", 'value': masked.data}, ignore_index=True)
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_vol_soil_lvl_2.pickle")

In [29]:
vol_soil_lvl_3_dir = os.path.join(dest_dir, "vol_soil_lvl_3")
file_list = os.listdir(vol_soil_lvl_3_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(vol_soil_lvl_3_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    dt = f[6:-4]
    date = find_year_and_month_datetime(dt, grace_dates)
    if date is not None:
        df = df.append({'date': date.strftime("%Y-%m-%d"), 'value': masked.data}, ignore_index=True)
        if date.year == 2012 and date.month == 1:
            df = df.append({'date': date.strftime("%Y-%m") + "-16", 'value': masked.data}, ignore_index=True)
        if date.year == 2015 and date.month == 4:
            df = df.append({'date': "2015-04-27", 'value': masked.data}, ignore_index=True)
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_vol_soil_lvl_3.pickle")

In [30]:
vol_soil_lvl_4_dir = os.path.join(dest_dir, "vol_soil_lvl_4")
file_list = os.listdir(vol_soil_lvl_4_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(vol_soil_lvl_4_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    dt = f[6:-4]
    date = find_year_and_month_datetime(dt, grace_dates)
    if date is not None:
        df = df.append({'date': date.strftime("%Y-%m-%d"), 'value': masked.data}, ignore_index=True)
        if date.year == 2012 and date.month == 1:
            df = df.append({'date': date.strftime("%Y-%m") + "-16", 'value': masked.data}, ignore_index=True)
        if date.year == 2015 and date.month == 4:
            df = df.append({'date': "2015-04-27", 'value': masked.data}, ignore_index=True)
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_vol_soil_lvl_4.pickle")